## LSTM-CNN Text Classification with Keras

In [1]:
%run Setup.ipynb

[i] Loaded Parameters:
 40000 30 0.2 25
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[i] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30542 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 4)
[+] Number of entries in each category:
[+] Training:
 [ 7699. 10910. 12239.  1152.]
[+] Validation:
 [1945. 2714. 3060.  281.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.25d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

### RNN Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(10,dropout=0.4,recurrent_dropout=0.4,return_sequences=True))(embedded_sequences)
l_cov1= Conv1D(25, 3, activation='relu',kernel_regularizer=regularizers.l2(0.02))(l_lstm1)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_drop1 = Dropout(0.4)(l_pool1)
#l_cov2 = Conv1D(32, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_drop1)
#l_pool2 = MaxPooling1D(2)(l_cov2)
#l_drop2 = Dropout(0.3)(l_pool2)
l_flat = Flatten()(l_drop1)
l_dense = Dense(25, activation='relu')(l_flat)
preds = Dense(4, activation='softmax')(l_dense)

In [4]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [5]:
def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.00001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)
    
def initial_boost(epoch):
    if epoch==0: return float(6.0)
    else: return float(1.0)
        
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=50, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoints", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#lr_schedule = callbacks.LearningRateScheduler(step_cyclic)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [6]:
model.summary()
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=32,
          callbacks=[tensorboard, model_checkpoints, lr_schedule])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 25)            763575    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 30, 20)            2880      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 28, 25)            1525      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 14, 25)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 25)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 350)               0         
__________

KeyboardInterrupt: 

In [ ]:
model.save('ltsm-c.h5')